In [1]:
from pathlib import Path
from dotenv import load_dotenv

import pandas as pd
import xgboost as xgb

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, f1_score, recall_score

import mlflow
from mlflow.models import infer_signature
import optuna
from optuna.integration.mlflow import MLflowCallback

load_dotenv()

/home/reezo/miniconda3/envs/fraud_detection/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
BASE_PATH = Path().resolve().parent
DATA_PATH = BASE_PATH / "data"
MODEL_PATH = BASE_PATH / "models"

In [3]:
df_train = pd.read_parquet(DATA_PATH / "processed" / "train_df.parquet")
df_val = pd.read_parquet(DATA_PATH / "processed" / "val_df.parquet")

In [9]:
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Shared/Fraud_detection")
1

1

In [5]:
X_train = df_train.drop(columns=["isFraud", "TransactionID"])
y_train = df_train["isFraud"]

X_val = df_val.drop(columns=["isFraud", "TransactionID"])
y_val = df_val["isFraud"]

### Baseline-model

In [6]:
CATALOG = "workspace"
SCHEMA = "default"
MODEL_NAME = "baseline-model"
FULL_MODEL_NAME = f"{CATALOG}.{SCHEMA}.{MODEL_NAME}"

mlflow.xgboost.autolog(
log_models=False,       
log_input_examples=False,
log_model_signatures=False,
)

with mlflow.start_run(run_name="test-xgb-var_thr-0.9",
                      tags={
                          "variance_Threshold": "0.9",
                          "experiment_type": "experimental",
                          }
                    ):

    model = xgb.XGBClassifier(eval_metric="auc")
    model.fit(X_train, y_train)
    
    params = model.get_params()
    mlflow.log_params(params)
    
    # Calculate ROC, metric for kaggle evaluation
    y_proba = model.predict_proba(X_val)[:, 1]  
    auc_score = roc_auc_score(y_val, y_proba)
    mlflow.log_metric("AUC_ROC", auc_score)

    mlflow.xgboost.log_model(
        xgb_model=model,
        name="baseline-model",
        input_example=X_train.head(),
        registered_model_name=FULL_MODEL_NAME,
    )

    y_pred = model.predict(X_val)

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)

    mlflow.log_metrics(
        {
            "accuracy": accuracy,
            "precision": precision,
            "f1": f1,
            "recall": recall
        }
    )

2025/12/06 21:13:07 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/reezo/miniconda3/envs/fraud_detection/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
/home/reezo/miniconda3/envs/fraud_detection/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred 

🏃 View run test-xgb-var_thr-0.9 at: https://dbc-a3a3bf55-bd05.cloud.databricks.com/ml/experiments/1577265081607397/runs/66a07b0a994e4ba09ae30328b28158ef
🧪 View experiment at: https://dbc-a3a3bf55-bd05.cloud.databricks.com/ml/experiments/1577265081607397


### Optimized-model

In [7]:
CATALOG = "workspace"
SCHEMA = "default"
MODEL_NAME = "optimized-model"
FULL_MODEL_NAME = f"{CATALOG}.{SCHEMA}.{MODEL_NAME}"
N_TRIALS = 50

def objective(trial):
    """
    Función objetivo que Optuna optimizará.
    Entrena un modelo con un conjunto de hiperparámetros
    y devuelve el score a maximizar.
    """
    params = {
        "objective": "binary:logistic",
        "tree_method": "hist",
        "eval_metric": "auc",
        "n_jobs": -1,
        "n_estimators": 5000,
        "early_stopping_rounds": 100,
        "verbosity": 0,
        
        "device": "cuda", ######### Change this if running on non nvidia hardware  ######

        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.05, log=True),
        "max_depth": trial.suggest_int("max_depth", 6, 15),
        "subsample": trial.suggest_float("subsample", 0.6, 0.9),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.9),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 10.0, log=True),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 10.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 100),
    }

    model = xgb.XGBClassifier(**params)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_val, y_val)], 
        verbose=False
    )

    y_proba = model.predict_proba(X_val)[:, 1]
    auc_score = roc_auc_score(y_val, y_proba)

    return auc_score

In [ ]:
mlflow_callback = MLflowCallback(
    tracking_uri=mlflow.get_tracking_uri(),
    metric_name="validation_AUC_ROC",
    create_experiment=False,
    mlflow_kwargs={"nested": True},
    )

with mlflow.start_run(run_name="optuna-xgb-tuning") as parent_run:
    print(f"Parent Run ID: {parent_run.info.run_id}")

    study = optuna.create_study(
        study_name="xgb-fraud-detection",
        direction="maximize",
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
    )

    study.optimize(
        objective,
        n_trials=N_TRIALS,
        callbacks=[mlflow_callback],
        show_progress_bar=True,
    )

    mlflow.log_metric("best_validation_AUC_ROC", study.best_value)
    mlflow.set_tag("best_trial_number", study.best_trial.number)


    best_params = {
        "objective": "binary:logistic",
        "tree_method": "hist",
        "eval_metric": "auc",
        "n_jobs": -1,
        "n_estimators": 5000,
        "early_stopping_rounds": 100,
        "verbosity": 0,
        **study.best_params,
        "device": "cuda", ######### Change this if running on non nvidia hardware  ######
    }

    final_model = xgb.XGBClassifier(**best_params)
    final_model.fit(
        X_train, 
        y_train,
        eval_set=[(X_val, y_val)], 
        verbose=False
    )

    y_proba_final = final_model.predict_proba(X_val)[:, 1]
    y_pred_final = final_model.predict(X_val)

    final_metrics = {
        "final_AUC_ROC": roc_auc_score(y_val, y_proba_final),
        "final_accuracy": accuracy_score(y_val, y_pred_final),
        "final_precision": precision_score(y_val, y_pred_final),
        "final_f1": f1_score(y_val, y_pred_final),
        "final_recall": recall_score(y_val, y_pred_final),
    }
    mlflow.log_metrics(final_metrics)

    signature = infer_signature(X_val, y_pred_final)

    mlflow.xgboost.log_model(
        xgb_model=final_model,
        artifact_path="best_model",
        signature=signature,
        input_example=X_train.head(5),
        registered_model_name=FULL_MODEL_NAME,
        )

    print(f"\n{'='*60}")
    print(f"Best AUC-ROC: {study.best_value:.6f}")
    print(f"Best params: {study.best_params}")
    print(f"Model registered as: {FULL_MODEL_NAME}")
    print(f"{'='*60}")

/tmp/ipykernel_63802/2834344188.py:1: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflow_callback = MLflowCallback(
[I 2025-12-06 21:13:32,658] A new study created in memory with name: xgb-fraud-detection


Parent Run ID: 06cf03f0f6ba42c4a44ee3831a841f63


  0%|          | 0/50 [00:55<?, ?it/s]

[I 2025-12-06 21:14:28,508] Trial 0 finished with value: 0.9179008822699205 and parameters: {'learning_rate': 0.005724812526567485, 'max_depth': 13, 'subsample': 0.8369039067359566, 'colsample_bytree': 0.8925327250306713, 'reg_alpha': 2.265238475438135, 'reg_lambda': 1.3962286796522771, 'scale_pos_weight': 6.988900632516041, 'min_child_weight': 52}. Best is trial 0 with value: 0.9179008822699205.


Best trial: 0. Best value: 0.917901:   2%|▏         | 1/50 [00:57<46:47, 57.31s/it]

...

[I 2025-12-06 21:54:18,441] Trial 49 finished with value: 0.9209425831819931 and parameters: {'learning_rate': 0.006533882543032923, 'max_depth': 13, 'subsample': 0.8287090653103164, 'colsample_bytree': 0.6360229535162575, 'reg_alpha': 0.1278345102516943, 'reg_lambda': 0.12502730337326, 'scale_pos_weight': 2.171671999659718, 'min_child_weight': 6}. Best is trial 48 with value: 0.9219228194552748.


Best trial: 48. Best value: 0.921923: 100%|██████████| 50/50 [40:47<00:00, 48.94s/it]

🏃 View run 49 at: https://dbc-a3a3bf55-bd05.cloud.databricks.com/ml/experiments/1577265081607397/runs/679e67656e10451a80474fc2e336837e
🧪 View experiment at: https://dbc-a3a3bf55-bd05.cloud.databricks.com/ml/experiments/1577265081607397



2025/12/06 21:54:51 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: The following failures occurred while performing one or more logging operations: [MlflowException("Failed to perform one or more operations on the run with ID 06cf03f0f6ba42c4a44ee3831a841f63. Failed operations: [RestException('INVALID_PARAMETER_VALUE: Parameter with key colsample_bytree was already logged with a value of 0.8925327250306713. The attempted new value was 0.6270359174095922')]")]
2025/12/06 21:55:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/reezo/miniconda3/envs/fraud_detection/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this


Mejor AUC-ROC: 0.921923
Mejores parámetros: {'learning_rate': 0.009684875454657554, 'max_depth': 13, 'subsample': 0.8230823949825079, 'colsample_bytree': 0.6270359174095922, 'reg_alpha': 0.1207132330884815, 'reg_lambda': 0.18680608424682976, 'scale_pos_weight': 2.108843679635887, 'min_child_weight': 6}
Modelo registrado como: workspace.default.optimized-model
🏃 View run optuna-xgb-tuning at: https://dbc-a3a3bf55-bd05.cloud.databricks.com/ml/experiments/1577265081607397/runs/06cf03f0f6ba42c4a44ee3831a841f63
🧪 View experiment at: https://dbc-a3a3bf55-bd05.cloud.databricks.com/ml/experiments/1577265081607397


In [10]:
y_proba_final = final_model.predict_proba(X_val)[:, 1]
y_pred_final = final_model.predict(X_val)
roc_auc_score(y_val, y_proba_final)

2025/12/06 22:08:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/reezo/miniconda3/envs/fraud_detection/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/12/06 22:08:19 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/reezo/miniconda3/en

0.9219228194552748